# Collecting Textual News Data from Twitter


## Importing Dependencies

In [1]:
from dotenv import load_dotenv
import os
import tweepy
import pandas as pd

## Setting Up Twitter API and Tokens from .env File

In [2]:
load_dotenv()

True

## Setting Up Twitter Screaper

In [3]:
tweets = []
userID = []
tweetID = []
tweet_url = []
username = []
next_page_token = []

In [4]:
def getClient():
    
    client = tweepy.Client(bearer_token= os.getenv("BEARER_TOKEN"),
                           consumer_key=os.getenv("API_KEY"), 
                           consumer_secret=os.getenv("API_KEY_SECRET"), 
                           access_token=os.getenv("ACCESS_TOKEN"),
                           access_token_secret=os.getenv("ACCESS_TOKEN_SECRET"))

    return client


def searchTweets(query, qty):
    
    client = getClient()
    client_call = client.search_all_tweets(query=query, max_results=qty, expansions='author_id')
    
    if not client_call.data is None and len(client_call.data) > 0: #Checking if there are results or not before looping...
        for tweet in client_call.data:
            tweets.append(tweet.text)
            id_ = tweet.id
            tweetID.append(id_)
            user_id = tweet.author_id
            userID.append(user_id)
            tweet_url.append('https://twitter.com/{}/status/{}'.format(user_id, id_))
            
    else:
        return ['NOOOOO DATA']
    
    try:
        next_page = client_call.meta['next_token']
        next_page_token.append(next_page)
    except KeyError:
        print("NO NEXT TOKEN AVAILABLE, THIS MEANS YOU GOT ONLY ONE PAGE.... :( ")

    df = pd.DataFrame({'Tweets': tweets, "Tweet URL": tweet_url, 'Tweet ID': tweetID, "User ID": userID})
           
    return df 


def search_tweets_next(query, qty, next_page_token, pagination):
    
    client = getClient()

    client_call_ = client.search_all_tweets(query=query, max_results=qty, expansions='author_id', next_token=next_page_token[pagination])
   
    if not client_call_.data is None and len(client_call_.data) > 0:
        for tweet in client_call_.data:
            tweets.append(tweet.text)
            id_ = tweet.id
            tweetID.append(id_)
            user_id = tweet.author_id
            userID.append(user_id)   
            tweet_url.append('https://twitter.com/{}/status/{}'.format(user_id, id_))
    else:
        return ['NOOOOO DATA']
    
    try:
        next_page = client_call_.meta['next_token']
        next_page_token.append(next_page)
    except KeyError:
        print("NO NEXT TOKEN AVAILABLE, THIS MEANS YOU GOT UNTIL THIS PAGE:( .... :( ")
    
    df = pd.DataFrame({'Tweets': tweets, "Tweet URL": tweet_url, 'Tweet ID': tweetID, "User ID": userID})
        
    return df


def get_user_by_id(userID):
    
    client = getClient()
    client_call = client.get_user(id=userID)
    user_id = client_call.data.username
    
    return user_id
    

# Calling Function to search for tweets
This function will get all tweets in a dataframe and then it will be exported on a CSV file

In [5]:
#First Data Request
NewsAccounts = ['factcheckdotorg', 'CNN', 'Forbes', 'BBCBreaking', 'BBC', 
                'nytimes', 'WSJ', 'washingtonpost', 'NewYorker', 
                'AP', 'Reuters', 'Bloomberg', 'ForeignAffairs', 
                'TheAtlantic', 'politico']

FakeNewsAccounts = ['TheOnion']

df_t = searchTweets('from:{} -is:retweet lang:en'.format(NewsAccounts[0]), 500)    


In [11]:
# df = pd.read_csv('Datasets/TrustedNews.csv')

In [13]:
# new_df = pd.concat([df, df_], axis=0)

(33295, 6)

In [6]:
#Data requests adding pagination token (each request gets 500 tweets, every single execuion I have to change the token manually)
pagination = 0
df_ = search_tweets_next('from:{} -is:retweet lang:en'.format(NewsAccounts[0]), 500, next_page_token, pagination)




NO NEXT TOKEN AVAILABLE, THIS MEANS YOU GOT UNTIL THIS PAGE:( .... :( 


(695, 4)

In [15]:
# .to_csv('Datasets/TrustedNews.csv')

df

In [ ]:
df = df_75.drop(df_75.index[df_75['User ID'] == 428333])

In [20]:
df.to_csv('Datasets/TrustedNews.csv')

(32600, 6)

(33295, 6)